In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

In [ ]:
!pip install -q decord

In [2]:
from transformers import AutoProcessor, AutoModelForCausalLM
import torch
device = "cuda:1"
processor = AutoProcessor.from_pretrained("microsoft/git-large-vatex", device=device, type=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained("microsoft/git-large-vatex", output_hidden_states=True, torch_dtype=torch.bfloat16).to(device)
# processor = AutoProcessor.from_pretrained("microsoft/git-large-msrvtt-qa", device=device, type=torch.bfloat16)
# model = AutoModelForCausalLM.from_pretrained("microsoft/git-large-msrvtt-qa", output_hidden_states=True, torch_dtype=torch.bfloat16).to(device)

# processor = AutoProcessor.from_pretrained("microsoft/git-large-vatex", device=device)
# model = AutoModelForCausalLM.from_pretrained("microsoft/git-large-vatex", output_hidden_states=True).to(device)

# processor = AutoProcessor.from_pretrained("microsoft/git-base-msrvtt-qa", device=device)
# model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-msrvtt-qa", output_hidden_states=True).to(device)

/opt/conda/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-02-11 12:11:01,083] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
model.dtype

torch.bfloat16

In [4]:
type(model)

transformers.models.git.modeling_git.GitForCausalLM

In [5]:
num_frames = model.config.num_image_with_embedding
print(num_frames)

6


In [6]:
import numpy as np
from decord import VideoReader, cpu
from torchvision import transforms
from einops import rearrange

def sample_frames(file_path, num_frames):
    # set seed for reproducability
    np.random.seed(45)

    def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
        converted_len = int(clip_len * frame_sample_rate)
        end_idx = np.random.randint(converted_len, seg_len)
        start_idx = end_idx - converted_len
        indices = np.linspace(start_idx, end_idx, num=clip_len)
        indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
        return indices

    # video clip consists of 300 frames (10 seconds at 30 FPS)
    videoreader = VideoReader(file_path, num_threads=1, ctx=cpu(0))

    # sample 6 frames
    videoreader.seek(0)
    indices = sample_frame_indices(clip_len=num_frames, frame_sample_rate=4, seg_len=len(videoreader))
    frames = videoreader.get_batch(indices).asnumpy()

    return list(frames)

def make_video_from_image(file_path, num_frames):
    image = Image.open(file_path)
    image.load()
    data = np.asarray(image)
    frames = [data] * num_frames
    # data = transforms.ToTensor()(image)
    # data = rearrange(data, 'c b h -> b h c')
    # # .permute(2, 0, 1)
    # frames = [data] * num_frames
    return frames


# video_path = "/research/video_metaphor/data/videos/6AR0xq6g5Uo.mp4"


/opt/conda/envs/llava/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/envs/llava/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [7]:
def sample_frames(file_path, num_frames):
    # set seed for reproducability
    np.random.seed(45)

    def get_frames_in_interval(start, end, number_of_frames):
        indices = np.random.randint(low = start, high = end, size = number_of_frames)
        indices = np.sort(indices)
        return indices


    def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
        # Making three intervals: Start, middle and the end
        interval_gap = seg_len // 3
        frames_per_gap = clip_len // 3
        
        begin_interval = get_frames_in_interval(0, interval_gap, frames_per_gap)
        middle_interval = get_frames_in_interval(interval_gap + 1, interval_gap * 2, frames_per_gap)
        end_interval = get_frames_in_interval((interval_gap*2) + 1, seg_len, frames_per_gap)
        frame_indices = np.concatenate((begin_interval, middle_interval, end_interval), axis = 0)
        print(frame_indices)
        return frame_indices
    


    # video clip consists of 300 frames (10 seconds at 30 FPS)
    videoreader = VideoReader(file_path, num_threads=1, ctx=cpu(0))

    # sample 6 frames
    videoreader.seek(0)
    indices = sample_frame_indices(clip_len=num_frames, frame_sample_rate=4, seg_len=len(videoreader))
    frames = videoreader.get_batch(indices).asnumpy()
    print(frames)

    return list(frames)

In [18]:
# Sampling multiple frames in each interval
def sample_frames_on_interval(file_path, num_frames):
    
    def get_frames_in_interval(start, end, number_of_frames):
        indices = np.random.randint(low = start, high = end, size = number_of_frames)
        indices = np.sort(indices)
        return indices


    # def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    #     # Making three intervals: Start, middle and the end
    #     interval_gap = seg_len // 3
    #     frames_per_gap = 6
        
    #     begin_interval = get_frames_in_interval(0, interval_gap, frames_per_gap)
    #     middle_interval = get_frames_in_interval(interval_gap + 1, interval_gap * 2, frames_per_gap)
    #     end_interval = get_frames_in_interval((interval_gap*2) + 1, seg_len, frames_per_gap)
    #     # frame_indices = np.concatenate((begin_interval, middle_interval, end_interval), axis = 0)
    #     # print(begin_interval)
    #     frames = [begin_interval, middle_interval, end_interval]
    #     print(frames)
    #     return frames

    def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
        # Making three intervals: Start, middle and the end
        interval_gap = seg_len // 16
        frames_per_gap = 6

        frames = []

        for i in range(0, 16):
            curr_interval = get_frames_in_interval(interval_gap * i, interval_gap * (i+1), frames_per_gap)
            print("Current interval: ",interval_gap * i, interval_gap * (i+1) )
            print("chosen frames: ")
            print(curr_interval)
            frames.append(curr_interval)
        
        
        print(frames)
        return frames
    
    videoreader = VideoReader(file_path, num_threads=1, ctx=cpu(0))

    # sample 6 frames
    videoreader.seek(0)
    indices = sample_frame_indices(clip_len=num_frames, frame_sample_rate=4, seg_len=len(videoreader))
    frame_set = []
    for index_set in indices:
        video_frames = videoreader.get_batch(index_set).asnumpy()
        frame_set.append(list(video_frames))

    return frame_set


In [19]:
video_path = "/research/video_metaphor/data/videos/L4mP9pR-mzU.mp4"

frames = sample_frames_on_interval(video_path, num_frames)
print(len(frames))


Current interval:  0 56
chosen frames: 
[ 7 16 19 21 36 45]
Current interval:  56 112
chosen frames: 
[ 69  75  94  95 106 108]
Current interval:  112 168
chosen frames: 
[112 122 124 135 137 166]
Current interval:  168 224
chosen frames: 
[178 180 197 213 219 221]
Current interval:  224 280
chosen frames: 
[239 244 244 246 268 273]
Current interval:  280 336
chosen frames: 
[292 305 309 327 329 335]
Current interval:  336 392
chosen frames: 
[338 347 352 362 365 368]
Current interval:  392 448
chosen frames: 
[392 396 399 400 408 442]
Current interval:  448 504
chosen frames: 
[453 461 463 465 470 476]
Current interval:  504 560
chosen frames: 
[517 518 528 539 542 543]
Current interval:  560 616
chosen frames: 
[560 564 571 574 577 591]
Current interval:  616 672
chosen frames: 
[640 657 660 662 663 671]
Current interval:  672 728
chosen frames: 
[675 676 687 710 723 726]
Current interval:  728 784
chosen frames: 
[731 751 754 759 772 779]
Current interval:  784 840
chosen frames: 
[

In [10]:
frames[0][0].shape

(360, 640, 3)

In [7]:
video_path = "/research/video_metaphor/data/videos/L4mP9pR-mzU.mp4"

frames = sample_frames(video_path, num_frames)
frames[0].shape

(360, 640, 3)

In [ ]:
from PIL import Image
img = Image.open("/research/video_metaphor/LLaVA/playground/data/irfl/metaphor_images/00afd9d5cce899ad9979c8ed9f42fc99_2.jpg")
img.load()
data = np.asarray(img)
print(data.shape)
frames = [data] * num_frames

In [ ]:
from PIL import Image
frames = make_video_from_image("/research/video_metaphor/LLaVA/playground/data/irfl/metaphor_images/00afd9d5cce899ad9979c8ed9f42fc99_2.jpg", num_frames)

In [8]:
frames[0].shape

(360, 640, 3)

In [9]:
frames[0].shape

(360, 640, 3)

In [ ]:
# device = "cpu"
# outputs = model(**inputs)

In [9]:
inputs['pixel_values'].shape

NameError: name 'inputs' is not defined

In [ ]:
torch.cuda.empty_cache()
torch.cuda.is_available()
torch.cuda.device_count()

In [ ]:
# video_path = "/research/video_metaphor/data/videos/L4mP9pR-mzU.mp4"

# frames = sample_frames(video_path, num_frames)
inputs = processor(images=frames, return_tensors="pt").to(device)

question = "What colour shirt is th lady wearing?"

input_ids = processor(text=question, add_special_tokens=False).input_ids
input_ids = [processor.tokenizer.cls_token_id] + input_ids
input_ids = torch.tensor(input_ids).unsqueeze(0)
input_ids = input_ids.to(device)#.to(torch.bfloat16)
with torch.no_grad():
    outputs = model(input_ids, pixel_values=inputs.pixel_values.to(device).to(torch.bfloat16))

In [11]:
# video_path = "/research/video_metaphor/data/videos/L4mP9pR-mzU.mp4"

# frames = sample_frames(video_path, num_frames)
inputs = processor(images=frames, return_tensors="pt").to(device)

input_ids = [processor.tokenizer.cls_token_id]
input_ids = torch.tensor(input_ids).unsqueeze(0)
input_ids = input_ids.to(device)#.to(torch.bfloat16)

with torch.no_grad():
    outputs = model(input_ids, pixel_values=inputs.pixel_values.to(device).to(torch.bfloat16))

/opt/conda/envs/llava/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:149: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  return torch.tensor(value)


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 0 but got size 1 for tensor number 1 in the list.

In [11]:
# Testing for training
inputs = processor(images=frames, text=["The guy is playing football"],return_tensors="pt").to(device)

print(inputs.keys())
# input_ids = [processor.tokenizer.cls_token_id]
# input_ids = torch.tensor(input_ids).unsqueeze(0)
# input_ids = input_ids.to(device)#.to(torch.bfloat16)

# with torch.no_grad():
#     outputs = model(input_ids, pixel_values=inputs.pixel_values.to(device).to(torch.bfloat16))

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])


In [16]:
# video_path = "/research/video_metaphor/data/videos/L4mP9pR-mzU.mp4"

# frames = sample_frames(video_path, num_frames)
inputs = processor(images=frames[5], return_tensors="pt").to(device)

input_ids = [processor.tokenizer.cls_token_id]
input_ids = torch.tensor(input_ids).unsqueeze(0)
input_ids = input_ids.to(device)#.to(torch.bfloat16)
generated_ids = model.generate(pixel_values=inputs.pixel_values.to(device).to(torch.bfloat16),input_ids=input_ids, max_length=50, output_hidden_states=True)
print("Generated caption:", processor.batch_decode(generated_ids, skip_special_tokens=True))
# with torch.no_grad():
#     outputs = model(input_ids, pixel_values=inputs.pixel_values.to(device).to(torch.bfloat16))

Generated caption: ['a woman is showing a snickers brand and then a snickers is shown.']


In [20]:
# Get hidden states for multiple parts of videos:
all_outputs = []


input_ids = [processor.tokenizer.cls_token_id]
input_ids = torch.tensor(input_ids).unsqueeze(0)
input_ids = input_ids.to(device)#.to(torch.bfloat16)

for frame in frames:
    inputs = processor(images=frame, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(input_ids, pixel_values=inputs.pixel_values.to(device).to(torch.bfloat16))
        hidden_states = outputs.hidden_states[-1].to(torch.bfloat16)
        all_outputs.append(hidden_states)

In [23]:
print(len(all_outputs))
print(type(all_outputs))
print(all_outputs[0].shape)
# all_hidden_states = torch.cat(all_outputs, dim=0)
all_hidden_states = sum(all_outputs)
# all_hidden_states = torch.div(all_hidden_states, 3)
print(all_hidden_states.shape)
print(all_hidden_states[0])

16
<class 'list'>
torch.Size([1, 1543, 768])
torch.Size([1, 1543, 768])
tensor([[  8.3750, -30.8750, -26.7500,  ...,  17.0000,  -7.0000,  -5.3125],
        [  8.8750, -13.1250, -10.8750,  ...,  14.1250,  14.3750,  -4.4688],
        [  8.8125, -10.0000, -10.9375,  ...,  12.8125,  17.5000,  -8.0000],
        ...,
        [  5.0000, -14.1250,   0.5234,  ...,   8.5625,  29.0000, -13.8750],
        [ 11.0000,  -4.5938,  -9.0000,  ...,  10.6250,  21.3750,  -4.4375],
        [ -1.9531, -21.2500, -35.0000,  ...,   3.7656,  10.7500, -26.6250]],
       device='cuda:1', dtype=torch.bfloat16)


In [27]:
x = torch.stack(all_outputs)
y = x.sum(dim=0)
y.shape
y

tensor([[[  8.3750, -30.8750, -26.6250,  ...,  17.1250,  -6.9688,  -5.3125],
         [  8.8750, -13.0625, -10.8125,  ...,  14.0625,  14.4375,  -4.4375],
         [  8.7500,  -9.9375, -10.8750,  ...,  12.8750,  17.5000,  -8.0625],
         ...,
         [  5.0000, -14.1250,   0.5078,  ...,   8.5625,  28.8750, -13.8125],
         [ 10.8750,  -4.5625,  -9.0625,  ...,  10.6250,  21.3750,  -4.4375],
         [ -1.9531, -21.2500, -34.7500,  ...,   3.7500,  10.7500, -26.7500]]],
       device='cuda:1', dtype=torch.bfloat16)

In [37]:
# Hidden size: 1190, 768]
t=outputs.hidden_states[-1].to(torch.bfloat16)
t.shape

torch.Size([1, 1543, 768])

In [12]:
results = [t, t]
results_tensor = torch.cat(results)

In [ ]:
results_tensor.shape

In [ ]:
forward_outputs = outputs.logits
forward_outputs.shape

In [53]:
generated_ids = model.generate(pixel_values=inputs.pixel_values.to(device).to(torch.bfloat16),input_ids=input_ids, max_length=50, output_hidden_states=True)

/opt/conda/envs/llava/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


In [54]:
print("Generated caption:", processor.batch_decode(generated_ids, skip_special_tokens=True))

Generated caption: ['a man is showing a snickers brand and then a woman is shown talking.']


In [ ]:
model.config

In [ ]:
## Checking if all video files can be loaded correctly

In [ ]:
file = open("/research/video_metaphor/LLaVA/playground/data/video_data/video300_train.json",'r')
dataset = json.load(file)
print(len(dataset))

In [ ]:
for data in dataset:
    file_path = data["video"]
    try:
        videoreader = VideoReader(file_path, num_threads=1, ctx=cpu(0))
    except:
        print("Failed for : ", file_path)


In [ ]:
/raid/nlp/abisekrk/abisek/research/video_metaphor/data/videos/
